In [1]:
using Profile, StatProfilerHTML, BenchmarkTools

In [2]:
using VALISTrader

In [3]:
using Base.Threads, SparseArrays

In [4]:
using
    Dates,
    Logging,
    Distributed,
    VALISTrader.Feature.Reader,
    VALISTrader.Feature.Encoding,
    VALISTrader.Commons.MultiProcessing,
    VALISTrader.Commons.MapSparse.MapMatrix,
    VALISTrader.Commons.CalcTools,
    VALISTrader.LSAUC.Stat

In [5]:
datadir = "C:/Users/msekino/workspace/data/SMN/encoder_feature_sample/data"
args_opts = Dict{String,String}(
    "featureheaderdir" => "C:/Users/msekino/workspace/data/SMN/encoder_feature_sample/data",
    "featurebodydir" => "C:/Users/msekino/workspace/data/SMN/encoder_feature_sample/data",
    "oldencoderfile" => "C:/Users/msekino/workspace/data/SMN/encoder_feature_sample/data/encoder_gllsauc.json"
)

Dict{String,String} with 3 entries:
  "featurebodydir"   => "C:/Users/msekino/workspace/data/SMN/encoder_feature_sa…
  "oldencoderfile"   => "C:/Users/msekino/workspace/data/SMN/encoder_feature_sa…
  "featureheaderdir" => "C:/Users/msekino/workspace/data/SMN/encoder_feature_sa…

In [6]:
headerfiles = findheaderfiles(args_opts["featureheaderdir"])
bodyfiles = findbodyfiles(args_opts["featurebodydir"])
columnnames = makecolumnnames(headerfiles...)
@info "makecolumnnames finished."

┌ Info: makecolumnnames finished.
└ @ Main In[6]:4


In [7]:
@time chunks = makechunks(headerfiles[1], bodyfiles; chunksize=20000)

 49.789629 seconds (95.76 M allocations: 9.697 GiB, 9.88% gc time)


Chunks(["E:\\temp\\jl_70B.tmp", "E:\\temp\\jl_209F.tmp", "E:\\temp\\jl_384F.tmp", "E:\\temp\\jl_4FDF.tmp", "E:\\temp\\jl_67AE.tmp", "E:\\temp\\jl_7F1F.tmp"])

In [8]:
encoder = loadoldfile(args_opts["oldencoderfile"], columnnames)
@info "loadoldfile finished."

┌ Info: loadoldfile finished.
└ @ Main In[8]:2


# @multiproc で for featurerow in chunk の空ループを回してみる

In [9]:
function calcmeansmultiproc(chunks::Chunks, encoder::Encoder)
    accumulator(
        x::Int64
    ) = begin
    end
    
    @multiproc (accumulator) for chunk in chunks
        for featurerow in chunk           
        end
        1
    end
end

calcmeansmultiproc (generic function with 1 method)

## nworkers() = 1

In [10]:
nworkers()

1

In [11]:
@time calcmeansmultiproc(chunks, encoder)

 13.584723 seconds (37.31 M allocations: 3.147 GiB, 16.12% gc time)


In [12]:
@time calcmeansmultiproc(chunks, encoder)

 14.722533 seconds (37.07 M allocations: 3.136 GiB, 23.12% gc time)


## nworkers() = 2

In [13]:
addprocs(2)

2-element Array{Int64,1}:
 2
 3

In [14]:
nworkers()

2

In [15]:
@time calcmeansmultiproc(chunks, encoder)

 23.601863 seconds (53.77 M allocations: 3.673 GiB, 14.80% gc time)


In [16]:
@time calcmeansmultiproc(chunks, encoder)

 23.373927 seconds (53.61 M allocations: 3.665 GiB, 14.94% gc time)


**むしろ遅くなった？！**

## nworkers() = 3

In [17]:
addprocs(1)

1-element Array{Int64,1}:
 4

In [18]:
nworkers()

3

In [19]:
@time calcmeansmultiproc(chunks, encoder)

 23.375105 seconds (53.61 M allocations: 3.665 GiB, 12.59% gc time)


In [20]:
@time calcmeansmultiproc(chunks, encoder)

 24.328640 seconds (53.61 M allocations: 3.665 GiB, 18.28% gc time)


# @threads で for featurerow in chunk の空ループを回してみる

In [21]:
function calcmeansmultithread(chunks::Chunks, encoder::Encoder)
    @threads for chunk in chunks
        for featurerow in chunk           
        end
    end
end

calcmeansmultithread (generic function with 1 method)

In [22]:
nthreads()

3

In [23]:
@time calcmeansmultithread(chunks, encoder)

  6.466924 seconds (37.30 M allocations: 3.141 GiB, 41.45% gc time)


**マルチスレッドが利いて速くなっているように見える**